In [1]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import re
import random
import numpy as np

okt = Okt()
komoran=Komoran()
kkma=Kkma()
hannanum=Hannanum()

In [2]:
df = pd.read_excel("/Users/young/git/project/export/비전공자 개발자_2023-01-31.xlsx")
df.rename(columns={'Unnamed: 0':'Rank'},inplace=True)


In [3]:
df.tail(5)

,Rank,title,description,author,timestamp
1045,1045,"문과 개발자, 정말 비전공자도 IT 취업이 가능할까?","문과 개발자, 정말 비전공자도 IT 취업이 가능할까? 문과라서 취업할 길이 보이...",그대 미소에 봄이 돼줄게요,2021.09.18.
1046,1046,"자바 개발자 되는 법, 비전공자에게 적합한 과정은?",자바 개발자 되는 법 비전공자에게 맞는 과정 자바 개발자가 되기 위해 비전공자에...,IT교육및취업전문가 융쌤,2022.06.03.
1047,1047,"소프트웨어개발자되는법, 컴공출신 아닌 비전공자 자바웹포폴...",오늘은 컴공전공생이 아닌 비전공자가 #소프트웨어개발자되는법 #웹개발프로젝트 제작...,월요일에 만든 블로그,2022.01.20.
1048,1048,종로자바학원에서 비전공자도 개발자 취업 가능해요!!,종로자바학원에서 비전공자도 개발자 취업 가능해요!! 비전공자분들의 가장 큰 고민...,따봉아빠의 IT 일상,2017.09.08.
1049,1049,(비전공자 공부순서) 개발자가 하고 싶어요.....,그럼에도 불구하고 디지털노마드를 꿈꾸시거나 개발자의 꿈을 갖고 계신 분 혹은.....,초보개발자 _ 왕곰,2022.09.13.


In [4]:
df['structure'] = ''

In [272]:
def pos_seperator(pos):
    #품사
    pos = re.search(('(?<=\/)[a-zA-Z]*'),tup)[0]
    #형태소
    morphs = re.search(('.+(?=\/)'),tup)[0]

def add_scores():
    return 

def word_parser(df):
    morphs_list = []
    phrase_list = []
    structure_list = []
    morphs_only_list = []
    # count_morphs_list = []
    # count_phrase_list = []
    # df 반복문 돌며 어절, 형태소 분해, 문장구조 분해 
    for idx,rows in df.iterrows():
        split_phrases = okt.phrases(rows[1])
        split_morphs = okt.pos(rows[1], join=True)
        morphs_tmp_list = []

        for i in split_phrases:
            phrase_list.append(i + '/' + 'Noun')
        for i in split_morphs:
            if 'Punctuation' in i:
                continue
            morphs_list.append(i)
            morphs_tmp_list.append(re.split(('/'),i)[0])
    #명사 중복 제거 후 구조화
        # tmp_structure = []
        tmp_structure = ''
        for jdx,tup in enumerate(okt.pos(rows[1])):
            if jdx > 0 and tmp == 'Noun' and tup[1] == 'Noun':
                continue
            tmp = okt.pos(rows[1])[jdx][1]
            # structure_list.append(tup[1])
            # tmp_structure += '_' + tup[1] if jdx != 0 or jdx != len(okt.pos(rows[1])) - 1 else tup[1]
            # if jdx != 0:
            #     tmp_structure += '_' + tup[1]
            # else:
            #     tmp_structure += tup[1]
            tmp_structure += '_' + tup[1]
            # structure_list.append([i for i in tup[1]])
        structure_list.append(tmp_structure)
        morphs_only_list.append(morphs_tmp_list)
    #unique값 찾기
    morphs_set = list(set(morphs_list))
    phrase_set = list(set(phrase_list))
    structure_set = list(set(structure_list))
    #빈도 카운트
    print(len(phrase_set))
    for i in range(len(morphs_set)):
        #두 리스트간 중복 삭제
        if morphs_set[i] in phrase_set:
            phrase_set.remove(morphs_set[i])
        morphs_set[i] = morphs_set[i] + '/' + str(morphs_list.count(morphs_set[i]))

    for i in range(len(phrase_set)):
        phrase_set[i] = phrase_set[i] + '_joint/' + str(phrase_list.count(phrase_set[i]))

    for i in range(len(structure_set)):
        structure_set[i] = structure_set[i] + '/' + str(structure_list.count(structure_set[i]))

    df['morphs'] = morphs_only_list
    return phrase_set,morphs_set,structure_set,structure_list

phrase_set,morphs_set,structure_set,structure_list = word_parser(df)



3251


In [6]:
df_orgin_structure = []
for i in structure_list:
    
    split = re.split(('_'),i)
    del split[0]

    df_orgin_structure.append(split)

df['structure'] = df_orgin_structure



In [7]:
#단어 토큰 df화
frame = {
    'word' : [],
    'pos' : [],
    'cnt' : []
}
word_st = phrase_set + morphs_set
for i in word_st:
    try:
        split = re.split(('/'),i)
        # print(split[1],split[2],split[3])
        frame['word'].append(split[0])
        frame['pos'].append(split[1])
        frame['cnt'].append(int(split[2]))
    except:
        # print(i)
        continue

frame
df2 = pd.DataFrame(frame)
df2 = df2.sort_values(by="cnt",ascending=False)
df2

,word,pos,cnt
2956,비,Noun,1063
3098,전공자,Noun,1062
3163,개발자,Noun,1029
306,비전공자,Noun_joint,659
3037,취업,Noun,599
...,...,...,...
1432,강남웹개발자학원 기초 강의,Noun_joint,1
1434,비전공자개발자취업 준비방법,Noun_joint,1
1435,소프트웨어개발 취업 후기,Noun_joint,1
1437,성공의 지름길,Noun_joint,1


In [8]:
# 문장구조 df화
structure_frame = {
    'structure' : [],
    'cnt' : []
}
for i in structure_set:
    split_cnt = re.split(('/'),i)
    cnt = int(split_cnt[-1])
    
    split = re.split(('_'),i)
    del split[0]
    split[-1] = re.split(('/'),split[-1])[0]
    # print(cnt)
    structure_frame['structure'].append(split)
    structure_frame['cnt'].append(cnt)
df3 = pd.DataFrame(structure_frame)
df3 = df3.sort_values(by="cnt",ascending=False)
df3

,structure,cnt
355,[Noun],40
216,"[Noun, Punctuation, Noun]",19
372,"[Noun, Josa, Noun]",18
260,"[Noun, Punctuation]",11
285,"[Noun, Punctuation, Noun, Punctuation]",9
...,...,...
284,"[Noun, Punctuation, Noun, Josa, Noun, Josa, Ve...",1
286,"[Noun, Josa, Noun, Punctuation, Noun, Punctuat...",1
287,"[Noun, Punctuation, Alpha, Noun, Josa, Noun, J...",1
288,"[Noun, Punctuation, Noun, Josa, Noun, Josa, Ve...",1


In [9]:
df3_pop = df3[df3['cnt'] > 1]

for i in range(1):
    sample_structure = random.choices(list(df3_pop['structure']),weights=[i for i in df3_pop['cnt']])[0]



In [10]:
sample_structure_dict = dict.fromkeys(sample_structure)

In [11]:
df2

,word,pos,cnt
2956,비,Noun,1063
3098,전공자,Noun,1062
3163,개발자,Noun,1029
306,비전공자,Noun_joint,659
3037,취업,Noun,599
...,...,...,...
1432,강남웹개발자학원 기초 강의,Noun_joint,1
1434,비전공자개발자취업 준비방법,Noun_joint,1
1435,소프트웨어개발 취업 후기,Noun_joint,1
1437,성공의 지름길,Noun_joint,1


In [41]:
#문장구조와 단어 조합
def random_choice(sample_list, weight):
    sample = random.choices(sample_list,weights=weight)
    return sample[0]

for pos in sample_structure:
    if pos == 'Noun':
        coin_flip = random_choice(["Noun","Noun_joint"],[0.2,0.8])
        df2_n = df2[df2['pos'] == coin_flip]
    else:
        df2_n = df2[df2['pos'] == pos]
    df2_n.reset_index(drop=False)
    df2_n.sort_values(by='cnt',ascending=False)

    match_senetence = []
    for i,row in df.iterrows():
        if sample_structure == row[5]:
            match_senetence.append(row[1])

    words = []
    
    for i in range(10):
        if pos == 'Punctuation':
            sample = random_choice(["?","!","??","!!"],[1,1,1,1])

        else:
            sample = random_choice(list(df2_n['word']),[i for i in df2_n['cnt']])

        words.append(sample)
    sample_structure_dict[pos] = words

print(sample_structure)
print(match_senetence)
print(sample_structure_dict)

['Noun', 'Punctuation', 'Noun', 'Verb', 'Verb', 'Adjective', 'Noun']
['코딩 개발자 - 비전공자 취업 준비전 읽어보면 좋은 글', '웹개발학원 , 비전공자 도 개발자 가 될수 있는 방법']
{'Noun': ['자바개발자부트캠프 비전공자취업대비반 6개월완성', '비전공자', '비전공자개발자', '취업 성공', '비전공자 취업', '서울아이티국비지원', '비전공자 프로그래밍', '비전공자', '개발자취업학원 비전공자', '비전공자 개발자 취업 전문 학원 추천'], 'Punctuation': ['?', '!', '!', '!!', '!!', '?', '??', '!!', '!!', '??'], 'Verb': ['될은', '되는', '만드는', '듣는', '모셔', '할', '할', '할', '하기', '하기'], 'Adjective': ['가능하다', '가능해', '가능할까', '가능한', '있을까', '가능해요', '뿐입니다', '있을까', '있는', '아님']}


In [42]:
tester = list(df['title'])
# tester = ['자바개발자취업 비전공자 교육 과정 안내!']

word_key = []
word_joint = []
for i in tester:
    x = 0
    pos = okt.pos(i)
    # word_joint.append([j[0] for j in pos])
    while True:
        words = ''
        for j in range(x,len(pos)):
            if pos[j][1] == 'Punctuation':
                continue
            words += pos[j][0]
            # if j == x:
            #     word_key.append(words)
            #     continue
            # else:
            #     word_joint.append(words)
            if j != x:
                word_key.append(pos[x][0])
                word_joint.append(words)
        x += 1
        if x == len(pos):
            break;
    
    
word_joint_aggregate = dict.fromkeys(word_key)
df_following_word = pd.DataFrame(word_joint,index=word_key,columns=['word'])

df_unique_counter = pd.DataFrame(df_following_word['word'].value_counts())
df_unique_counter.rename(columns={"word":"count"},inplace=True)
df_unique_counter['word'] = df_unique_counter.index
df_following_word['key'] = df_following_word.index
df_following_word.drop_duplicates(inplace=True)
df_word_joint = pd.merge(df_following_word,df_unique_counter,left_on='word',right_on='word',how='inner')
df_word_joint = df_word_joint[['key','word','count']]

df_word_joint


,key,word,count
0,비,비전공자,1370
1,:,비전공자,1370
2,-,비전공자,1370
3,",",비전공자,1370
4,(,비전공자,1370
...,...,...,...
42823,),개발자가하고싶어요,2
42824,개발자,개발자가하고싶어요,2
42825,가,가하고,1
42826,가,가하고싶어요,1


In [238]:
unique_starter = list(set(list(df_word_joint['key'])))
df_odds = pd.DataFrame()
num = 0

for starter in unique_starter:

    df_pieces = df_word_joint[df_word_joint['key'] == starter]
    df_pieces['odds'] = df_pieces['count']/df_pieces['count'].sum()

    if num != 0:
        df_odds = pd.concat([df_odds,df_pieces])
    else:
        df_odds = df_pieces
    num += 1


/Users/young/opt/anaconda3/envs/test1/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [249]:
starter = '문과'
tmp_forward = []

for i in range (15):
    if True not in list(df_word_joint['key'].isin([starter])):
        starter = okt.morphs(starter)[-1]
        if True not in list(df_word_joint['key'].isin([starter])):
            starter = starter.strip()[-1]

    df_draw = df_odds[df_odds['key'] == starter].sort_values(by='odds',ascending=False)

    draw_words = list(df_draw.head(5).iloc[:,1])
    draw_odds = list(df_draw.head(5).iloc[:,3])
    draw = random_choice(draw_words,draw_odds)

    if len(tmp_forward) == 0:
        tmp_forward.append(starter)
    if len(tmp_forward) > 0:
        element = draw.lstrip(tmp_forward[-1])
        if element == '':
            tmp_forward.append(draw)
        else:
            tmp_forward.append(element)
    # tmp_forward.append(draw)

    starter = draw.lstrip(starter)

tmp_forward

['문과',
 '생',
 '이현실적으로개발자직무전환',
 '할',
 '수',
 '있을까',
 '취업시장과실무',
 '교육',
 '비전공자',
 '개발자취업',
 '비전공자',
 '개발자취업',
 '을',
 '위',
 '한',
 '공부']

거리 계산 단계. 발사된 포탄이 어떤 군집과 가장 가까운가?

In [266]:
# df['morphs'] = okt.morphs(df['title'])
a = okt.pos("거리 계산 단계!",join=True)
re.split(('/'),a[1])[0]


'계산'

In [324]:
df

,Rank,title,description,author,timestamp,structure,morphs
0,0,비전공자 개발자 : 준비하기 적합한 케이스 및 취업률 높이기,"직업, 개발자를 목표로 삼는 경우가 많습니다. 하지만, 비전공자 개발자 취업이 ...",주들의 그리 대단하진 않지만 굉장한 이야기,어제,"[Noun, Punctuation, Noun, Verb, Adjective, Nou...","[비, 전공자, 개발자, 준비, 하기, 적합한, 케이스, 및, 취업률, 높이기]"
1,1,코딩 개발자 취업 비전공자의 가능성과 연봉 알아보기,등등 비전공자 개발자로 취업을 준비할 때 가장 중요한 건 직무 선택입니다. 코딩...,인공지능 특이점이 찾아올거 같아서 만든 블로그,2023.01.02.,"[Noun, Josa, Noun, Josa, Noun, Verb]","[코딩, 개발자, 취업, 비, 전공자, 의, 가능성, 과, 연봉, 알아보기]"
2,2,코딩 개발자 비전공자 취업 현실,비전공자 코딩 개발자 취업 현실 - 초봉 2600만 원 - 파견직 개발자 단기간...,인테리어,5시간 전,[Noun],"[코딩, 개발자, 비, 전공자, 취업, 현실]"
3,3,"자바개발자 : 비전공자, 취업성공의 지름길","자바개발자 : 비전공자, 취업성공의 지름길 IT개발자 중 가장 많은 수요를 기록...",비비디 바비디 부,2022.12.07.,"[Noun, Punctuation, Noun, Punctuation, Noun, J...","[자바, 개발자, 비, 전공자, 취업, 성공, 의, 지름길]"
4,4,비전공자개발자 학원에서 취업성공한 후기,"있는 비전공자분들 비전공자개발자 취업을 준비한다면, 이미 비슷한 조건과 상황.....",할일은 없지만 놀기만하지않ㅇ,2022.12.08.,"[Noun, Josa, Noun, Adjective, Noun]","[비, 전공자, 개발자, 학원, 에서, 취업, 성공한, 후기]"
...,...,...,...,...,...,...,...
1045,1045,"문과 개발자, 정말 비전공자도 IT 취업이 가능할까?","문과 개발자, 정말 비전공자도 IT 취업이 가능할까? 문과라서 취업할 길이 보이...",그대 미소에 봄이 돼줄게요,2021.09.18.,"[Noun, Punctuation, Noun, Josa, Alpha, Noun, J...","[문과, 개발자, 정말, 비, 전공자, 도, IT, 취업, 이, 가능할까]"
1046,1046,"자바 개발자 되는 법, 비전공자에게 적합한 과정은?",자바 개발자 되는 법 비전공자에게 맞는 과정 자바 개발자가 되기 위해 비전공자에...,IT교육및취업전문가 융쌤,2022.06.03.,"[Noun, Verb, Noun, Punctuation, Noun, Josa, Ad...","[자바, 개발자, 되는, 법, 비, 전공자, 에게, 적합한, 과정, 은]"
1047,1047,"소프트웨어개발자되는법, 컴공출신 아닌 비전공자 자바웹포폴...",오늘은 컴공전공생이 아닌 비전공자가 #소프트웨어개발자되는법 #웹개발프로젝트 제작...,월요일에 만든 블로그,2022.01.20.,"[Noun, Verb, Noun, Punctuation, Noun, Adjectiv...","[소프트웨어, 개발자, 되는, 법, 컴공, 출신, 아닌, 비, 전공자, 자바, 웹포폴]"
1048,1048,종로자바학원에서 비전공자도 개발자 취업 가능해요!!,종로자바학원에서 비전공자도 개발자 취업 가능해요!! 비전공자분들의 가장 큰 고민...,따봉아빠의 IT 일상,2017.09.08.,"[Noun, Josa, Noun, Josa, Noun, Adjective, Punc...","[종로, 자바, 학원, 에서, 비, 전공자, 도, 개발자, 취업, 가능해요]"


In [ ]:
#tf idf 계산

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
title = []
for i in list(df['title']):
    title.append(i)

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(title)
word2id = defaultdict(lambda : 0)

for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(title):
    print('====== document[%d] ======' % i)
    print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )
    print(sent)

In [330]:
from sklearn.metrics.pairwise import cosine_similarity

countervect = sp_matrix.toarray()
countetvect_df = pd.DataFrame(countervect,columns=sorted(vectorizer.vocabulary_))
countetvect_df.index = df.index

cosine_similarity(countetvect_df,countetvect_df)

array([[1.        , 0.01536182, 0.04509328, ..., 0.01162687, 0.0189158 ,
        0.01278535],
       [0.01536182, 1.        , 0.27555628, ..., 0.        , 0.06548491,
        0.        ],
       [0.04509328, 0.27555628, 1.        , ..., 0.02178428, 0.10358263,
        0.02395483],
       ...,
       [0.01162687, 0.        , 0.02178428, ..., 1.        , 0.        ,
        0.01339404],
       [0.0189158 , 0.06548491, 0.10358263, ..., 0.        , 1.        ,
        0.        ],
       [0.01278535, 0.        , 0.02395483, ..., 0.01339404, 0.        ,
        1.        ]])